Endspoints:

In [6]:
import pandas as pd
import gzip
import json
import ast
import re
import random

In [7]:
ruta_archivo = 'steam_games.csv'
df = pd.read_csv(ruta_archivo)

In [8]:
ruta_archivo = 'user_reviews.csv'
df2 = pd.read_csv(ruta_archivo)

In [9]:
ruta_archivo = 'user_items.csv'
df3 = pd.read_csv(ruta_archivo)

C:\Users\COMPUTER\AppData\Local\Temp\ipykernel_3868\2666023782.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv(ruta_archivo)


1- def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [5]:
def PlayTimeGenre(genero: str):
    
    df['User_Id'] = df['User_Id'].astype(str)
    df3['User_Id'] = df3['User_Id'].astype(str)
    # Fusionar ambos DataFrames en base a la columna 'user_id'
    df_combinado = df.merge(df3, left_on='User_Id', right_on='User_Id', how='inner')
    df_filtrado = df_combinado[df_combinado['Genres'].str.contains(genero, case=False, na=False)]

    if df_filtrado.empty:
        return {}  

    
    agrupado = df_filtrado.groupby('Release_Year')['Playtime_Forever'].sum()
    max_year = agrupado.idxmax()
    resultado = {"Año de lanzamiento con más horas jugadas para Género " + genero: max_year}

    return resultado

In [6]:
PlayTimeGenre('Indie')


{'Año de lanzamiento con más horas jugadas para Género Indie': 2014}

2-def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [7]:
def UserForGenre(genero: str):
    
    df['User_Id'] = df['User_Id'].astype(str)
    df3['User_Id'] = df3['User_Id'].astype(str)
    
    # Fusionar ambos DataFrames en base a la columna 'user_id'
    df_combinado = df.merge(df3, left_on='User_Id', right_on='User_Id', how='inner')

    df_filtrado = df_combinado[df_combinado['Genres'].str.contains(genero, case=False, na=False)]
    if df_filtrado.empty:
        return {}  
    agrupado = df_filtrado.groupby(['User_Id', 'Release_Year'])['Playtime_Forever'].sum().reset_index()
    max_user = agrupado.groupby('User_Id')['Playtime_Forever'].sum().idxmax()
    df_max_user = agrupado[agrupado['User_Id'] == max_user]
    lista_acumulacion = df_max_user.rename(columns={'Release_Year': 'Año', 'Playtime_Forever': 'Horas'}).to_dict('records')
    resultado = {
        "Usuario con más horas jugadas para el Género " + genero: max_user,
        "Horas jugadas": lista_acumulacion
    }

    return resultado

In [8]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para el Género Indie': '305010',
 'Horas jugadas': [{'User_Id': '305010', 'Año': 2014, 'Horas': 59519}]}

3-def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [13]:
def UsersRecommend(año: int):
    # filas para el año  y donde recommend es False
    juegos_no_recomendados = df2[(df2['Recommend'] == True)]
    
    # muestra aleatoria de df3
    sample_size = 90000  
    df3_sample = df3.sample(n=sample_size, random_state=42)
    
    
    juegos_no_recomendados = juegos_no_recomendados.merge(df3_sample[['Item_Id', 'Item_Name']], on='Item_Id', how='left')
    conteo_juegos = juegos_no_recomendados['Item_Name'].value_counts().reset_index()
    conteo_juegos.columns = ['Item_Name', 'count']
    top_juegos_no_recomendados = conteo_juegos.sort_values(by='count', ascending=True).head(3)
    
    resultado = [{"Puesto {}: {}".format(i+1, juego)} for i, juego in enumerate(top_juegos_no_recomendados['Item_Name'])]
    
    return resultado


In [14]:
UsersRecommend(2014)

[{'Puesto 1: Putt-Putt Goes to the Moon'},
 {'Puesto 2: Riven'},
 {'Puesto 3: Jagged Alliance Flashback'}]

4-def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

   Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [13]:
def UsersNotRecommend(año: int):
     #filas para el año dado y donde recommend es False
    juegos_no_recomendados = df2[(df2['Recommend'] == False)]
    
    juegos_no_recomendados = juegos_no_recomendados.merge(df3[['Item_Id', 'Item_Name']], on='Item_Id', how='left')
    conteo_juegos = juegos_no_recomendados['Item_Name'].value_counts().reset_index()
    conteo_juegos.columns = ['Item_Name', 'count']
    
    top_juegos_no_recomendados = conteo_juegos.sort_values(by='count', ascending=True).head(3)
    
    
    resultado = [{"Puesto {}: {}".format(i+1, juego)} for i, juego in enumerate(top_juegos_no_recomendados['Item_Name'])]
    
    return resultado



In [14]:
UsersNotRecommend(2011)


[{'Puesto 1: Earth: Year 2066'},
 {'Puesto 2: Avaris 2'},
 {'Puesto 3: World of Subways 1 – The Path'}]

5-def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

   Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [15]:
def sentiment_analysis(anio: int):
    # Filtra el DataFrame para obtener solo las filas correspondientes al año especificado
    df_filtrado = df2[df2['Year'] == anio]

    # Contabiliza la cantidad de registros en cada categoría de análisis de sentimiento
    categorias_sentimiento = df_filtrado['Sentiment_Analysis'].value_counts()

    # Crea un diccionario con el formato deseado
    resultado = {
        'Negativo': categorias_sentimiento.get(0, 0),  # Si no hay registros negativos, se establece en 0
        'Neutral': categorias_sentimiento.get(1, 0),   # Si no hay registros neutrales, se establece en 0
        'Positivo': categorias_sentimiento.get(2, 0)   # Si no hay registros positivos, se establece en 0
    }

    return resultado

In [16]:
# Ejemplo de uso de la función
anio = 2014  # Cambia el año según tus necesidades
resultado = sentiment_analysis(anio)
print(resultado)

{'Negativo': 1830, 'Neutral': 13358, 'Positivo': 6923}


In [ ]:
https://apirest-aaex.onrender.com/docs